# Определение стоимости автомобилей

Сервис по продаже автомобилей с пробегом «Не бит, не крашен» разрабатывает приложение для привлечения новых клиентов. В нём можно быстро узнать рыночную стоимость своего автомобиля. В нашем распоряжении исторические данные: технические характеристики, комплектации и цены автомобилей. Вам нужно построить модель для определения стоимости. 

Заказчику важны:

- качество предсказания;
- скорость предсказания;
- время обучения.

### Описание данных
Данные находятся в файле /datasets/autos.csv. Скачать датасет. 

Признаки
- DateCrawled — дата скачивания анкеты из базы
- VehicleType — тип автомобильного кузова
- RegistrationYear — год регистрации автомобиля
- Gearbox — тип коробки передач
- Power — мощность (л. с.)
- Model — модель автомобиля
- Kilometer — пробег (км)
- RegistrationMonth — месяц регистрации автомобиля
- FuelType — тип топлива
- Brand — марка автомобиля
- Repaired — была машина в ремонте или нет
- DateCreated — дата создания анкеты
- NumberOfPictures — количество фотографий автомобиля
- PostalCode — почтовый индекс владельца анкеты (пользователя)
- LastSeen — дата последней активности пользователя

Целевой признак
- Price — цена (евро)

## Подготовка данных

In [1]:
#!pip install lightgbm
#!pip install xgboost
#!pip install catboost
#!pip install category_encoders
#!pip install imbalanced-learn
#!pip install feature-engine
#!pip install imblearn

In [3]:
import warnings
warnings.filterwarnings('ignore')


import pandas as pd
import numpy as np

from sklearn.pipeline import Pipeline, make_pipeline

from category_encoders import TargetEncoder
from feature_engine.selection import DropCorrelatedFeatures, DropConstantFeatures

from sklearn.model_selection import train_test_split
from sklearn.compose import make_column_transformer 
from sklearn.preprocessing import StandardScaler
from sklearn.impute import KNNImputer
from sklearn.preprocessing import PolynomialFeatures
from sklearn.ensemble import VotingRegressor

from sklearn.metrics import mean_squared_error

from sklearn.tree import DecisionTreeRegressor
from lightgbm import LGBMRegressor
from catboost import CatBoostRegressor

from sklearn.model_selection import RandomizedSearchCV

In [4]:
try:
    data=pd.read_csv('autos.csv')
except:
    data=pd.read_csv('/datasets/autos.csv')


In [5]:
print('DataFrame: компании  «Не бит, не крашен»')
display(data.head(5))
print('')
print('Информация от DataFrame ')
print('')
data.info()
print('')
print('Информация от статистических данных по DataFrame')
print('')
display(data.describe())
print('')
print('Информация о количестве пропусков в DataFrame ')
print('')
print(data.isna().sum())
print('')
print('Информация о количестве дубликатов в DataFrame ')
print(data.duplicated().sum())
print('')
print('Информация о уникальных значениях в DataFrame ')
for i in (data):
    print('')

    print(f'Уникальные значения столбца {i}')
    print(data[i].unique())  

DataFrame: компании  «Не бит, не крашен»


,DateCrawled,Price,VehicleType,RegistrationYear,Gearbox,Power,Model,Kilometer,RegistrationMonth,FuelType,Brand,Repaired,DateCreated,NumberOfPictures,PostalCode,LastSeen
0,2016-03-24 11:52:17,480,NaN,1993,manual,0,golf,150000,0,petrol,volkswagen,NaN,2016-03-24 00:00:00,0,70435,2016-04-07 03:16:57
1,2016-03-24 10:58:45,18300,coupe,2011,manual,190,NaN,125000,5,gasoline,audi,yes,2016-03-24 00:00:00,0,66954,2016-04-07 01:46:50
2,2016-03-14 12:52:21,9800,suv,2004,auto,163,grand,125000,8,gasoline,jeep,NaN,2016-03-14 00:00:00,0,90480,2016-04-05 12:47:46
3,2016-03-17 16:54:04,1500,small,2001,manual,75,golf,150000,6,petrol,volkswagen,no,2016-03-17 00:00:00,0,91074,2016-03-17 17:40:17
4,2016-03-31 17:25:20,3600,small,2008,manual,69,fabia,90000,7,gasoline,skoda,no,2016-03-31 00:00:00,0,60437,2016-04-06 10:17:21



Информация от DataFrame 

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 354369 entries, 0 to 354368
Data columns (total 16 columns):
 #   Column             Non-Null Count   Dtype 
---  ------             --------------   ----- 
 0   DateCrawled        354369 non-null  object
 1   Price              354369 non-null  int64 
 2   VehicleType        316879 non-null  object
 3   RegistrationYear   354369 non-null  int64 
 4   Gearbox            334536 non-null  object
 5   Power              354369 non-null  int64 
 6   Model              334664 non-null  object
 7   Kilometer          354369 non-null  int64 
 8   RegistrationMonth  354369 non-null  int64 
 9   FuelType           321474 non-null  object
 10  Brand              354369 non-null  object
 11  Repaired           283215 non-null  object
 12  DateCreated        354369 non-null  object
 13  NumberOfPictures   354369 non-null  int64 
 14  PostalCode         354369 non-null  int64 
 15  LastSeen           354369 non-null  objec

,Price,RegistrationYear,Power,Kilometer,RegistrationMonth,NumberOfPictures,PostalCode
count,354369.000000,354369.000000,354369.000000,354369.000000,354369.000000,354369.0,354369.000000
mean,4416.656776,2004.234448,110.094337,128211.172535,5.714645,0.0,50508.689087
std,4514.158514,90.227958,189.850405,37905.341530,3.726421,0.0,25783.096248
min,0.000000,1000.000000,0.000000,5000.000000,0.000000,0.0,1067.000000
25%,1050.000000,1999.000000,69.000000,125000.000000,3.000000,0.0,30165.000000
50%,2700.000000,2003.000000,105.000000,150000.000000,6.000000,0.0,49413.000000
75%,6400.000000,2008.000000,143.000000,150000.000000,9.000000,0.0,71083.000000
max,20000.000000,9999.000000,20000.000000,150000.000000,12.000000,0.0,99998.000000



Информация о количестве пропусков в DataFrame 

DateCrawled              0
Price                    0
VehicleType          37490
RegistrationYear         0
Gearbox              19833
Power                    0
Model                19705
Kilometer                0
RegistrationMonth        0
FuelType             32895
Brand                    0
Repaired             71154
DateCreated              0
NumberOfPictures         0
PostalCode               0
LastSeen                 0
dtype: int64

Информация о количестве дубликатов в DataFrame 
4

Информация о уникальных значениях в DataFrame 

Уникальные значения столбца DateCrawled
['2016-03-24 11:52:17' '2016-03-24 10:58:45' '2016-03-14 12:52:21' ...
 '2016-03-21 09:50:58' '2016-03-14 17:48:27' '2016-03-19 18:57:12']

Уникальные значения столбца Price
[  480 18300  9800 ... 12395 18429 10985]

Уникальные значения столбца VehicleType
[nan 'coupe' 'suv' 'small' 'sedan' 'convertible' 'bus' 'wagon' 'other']

Уникальные значения столбца Registra

['2016-04-07 03:16:57' '2016-04-07 01:46:50' '2016-04-05 12:47:46' ...
 '2016-03-19 20:44:43' '2016-03-29 10:17:23' '2016-03-21 10:42:49']


In [6]:
data.drop_duplicates(inplace=True)

Так как цены на автомобили меняются ежегодно рассмотрим данные только за  последний год.

In [7]:
data['DateCreated'] = pd.to_datetime(data['DateCreated'], format='%Y-%m-%d %H:%M:%S')

In [8]:
data.shape

(354365, 16)

In [9]:
data_actual = data.drop(data[data['DateCreated'].dt.year < 2016].index)

In [10]:
data_actual.shape

(354339, 16)

После первичного анализа данных исключим не целевые признаки из таблицы.

In [11]:
data_features=data_actual.loc[:,'Price':'Repaired']
display(data_features)

,Price,VehicleType,RegistrationYear,Gearbox,Power,Model,Kilometer,RegistrationMonth,FuelType,Brand,Repaired
0,480,NaN,1993,manual,0,golf,150000,0,petrol,volkswagen,NaN
1,18300,coupe,2011,manual,190,NaN,125000,5,gasoline,audi,yes
2,9800,suv,2004,auto,163,grand,125000,8,gasoline,jeep,NaN
3,1500,small,2001,manual,75,golf,150000,6,petrol,volkswagen,no
4,3600,small,2008,manual,69,fabia,90000,7,gasoline,skoda,no
...,...,...,...,...,...,...,...,...,...,...,...
354364,0,NaN,2005,manual,0,colt,150000,7,petrol,mitsubishi,yes
354365,2200,NaN,2005,NaN,0,NaN,20000,1,NaN,sonstige_autos,NaN
354366,1199,convertible,2000,auto,101,fortwo,125000,3,petrol,smart,no
354367,9200,bus,1996,manual,102,transporter,150000,3,gasoline,volkswagen,no


Исключим из данных все объявления со стоимостью менее 250 евро, так как данная цена менее цены утилизации автомобиля.

In [12]:
(data_features['Price'] <250).sum()

19991

In [13]:
data_features_price = data_features.drop(data_features[(data_features['Price'] <250) ]  .index)


In [14]:
data_features_price.sort_values(by='Price',  ascending = True).head(5)

,Price,VehicleType,RegistrationYear,Gearbox,Power,Model,Kilometer,RegistrationMonth,FuelType,Brand,Repaired
186034,250,wagon,1996,NaN,0,mondeo,150000,11,petrol,ford,NaN
10395,250,small,1997,manual,50,fiesta,150000,11,petrol,ford,yes
136011,250,wagon,1998,manual,115,mondeo,20000,9,petrol,ford,NaN
136056,250,small,1996,manual,60,corsa,150000,6,petrol,opel,NaN
136118,250,small,1989,manual,54,micra,150000,9,petrol,nissan,NaN


Исключим из данных все объявления с регистаристарцией автомобилей до 1990 года и позже 2016.

In [15]:
data_features_price_year = data_features_price.drop(data_features_price[(data_features_price['RegistrationYear'] > 2016) | (data_features_price['RegistrationYear'] < 1990)].index)

Исключим из данных объявления в которых мощность автомобиля  менее 50лс и выше 500 лс. 

In [16]:
data_features_price_years_power = data_features_price_year.drop(data_features_price_year[(data_features_price_year['Power'] > 500) | (data_features_price_year['Power'] < 50)].index)

Приведем значения 'gasoline', 'petrol' к одному значению petrol так как все это является бензином.

In [17]:
data_features_price_years_power['FuelType'] = data_features_price_years_power['FuelType'].replace('gasoline', 'petrol')

Приведем значения 'land_rover' к 'rover' к rover так как land rover это модель марки 'rover'.

In [18]:
data_features_price_years_power['Brand'] = data_features_price_years_power['Brand'].replace('land_rover', 'rover')

Отсутствующие данные в  Repaired заменим на 'unknown'.

In [19]:
data_features_price_years_power['Repaired'] = data_features_price_years_power['Repaired'].fillna('unknown')
data_features_final = data_features_price_years_power

In [20]:
display(data_features_final.head(5))

,Price,VehicleType,RegistrationYear,Gearbox,Power,Model,Kilometer,RegistrationMonth,FuelType,Brand,Repaired
1,18300,coupe,2011,manual,190,NaN,125000,5,petrol,audi,yes
2,9800,suv,2004,auto,163,grand,125000,8,petrol,jeep,unknown
3,1500,small,2001,manual,75,golf,150000,6,petrol,volkswagen,no
4,3600,small,2008,manual,69,fabia,90000,7,petrol,skoda,no
5,650,sedan,1995,manual,102,3er,150000,10,petrol,bmw,yes


## Выводы о подготовке данных

В ходе подготовки данных был рассмотрен dataframe компании  «Не бит, не крашен».
После первичного анализа данных исключили  не целевые признаки DateCreated, NumberOfPictures, PostalCode, LastSeen из таблицы.
Исключили из данных все объявления со стоимостью менее 250 евро, так как данная цена менее цены утилизации автомобиля.
Исключили из данных все объявления с регистаристарцией автомобилей до 1990 года и позже 2016.
Исключили из данных объявления в которых мощность  автомобиля  менее 50лс и выше 500 лс. 
Привели значения 'gasoline', 'petrol' к одному значению 'petrol', так как все это является бензином.
Приведем значения 'land_rover' к 'rover' к 'rover' так как 'land rover' - это модель марки 'rover'.
Отсутствующие данные в  Repaired заменим на 'unknown' так как не возможно восстановить информацию по данному параметру. 

## Обучение моделей

Подготовим обучающие и целевые признаки.

In [21]:
features = data_features_final.drop(['Price'], axis=1)

target = data_features_final['Price']


Разделим данные на тренировочную и  тестовую выборку.

In [22]:
features_train, features_test, target_train, target_test =train_test_split (features, target, test_size=0.25,random_state=12345)


In [23]:
display(features_train.head(5))

features_train.shape

,VehicleType,RegistrationYear,Gearbox,Power,Model,Kilometer,RegistrationMonth,FuelType,Brand,Repaired
281301,convertible,2003,manual,163,slk,80000,6,petrol,mercedes_benz,no
235907,small,2001,manual,50,lupo,150000,8,petrol,volkswagen,no
77283,small,2009,manual,65,micra,40000,9,petrol,nissan,no
307199,sedan,2003,manual,105,golf,150000,11,petrol,volkswagen,no
144773,small,2006,manual,80,fiesta,150000,7,petrol,ford,no


(210152, 10)

In [24]:
display(target.head(5))

target_train.shape

1    18300
2     9800
3     1500
4     3600
5      650
Name: Price, dtype: int64

(210152,)

Выделим категориальные и числовые признаки 

In [25]:
cat_colunmns = features_train.select_dtypes(include=['object', 'category']).columns.tolist()

cat_colunmns

['VehicleType', 'Gearbox', 'Model', 'FuelType', 'Brand', 'Repaired']

In [26]:
num_columns = features_train.select_dtypes(include=['int64']).columns.tolist()

num_columns

['RegistrationYear', 'Power', 'Kilometer', 'RegistrationMonth']

Подготовим 'pipeline':
Выполним масштабирование признаков:
Исключим признаки  мультиколинеарные признаки.
Используя библиотеку KNNImputer заполним категориальные признаки методом ближайших соседей 
Выполним генерацию полиномиальных признаков для лучшего поиска нелинейных связей данных.

In [27]:
scaler = StandardScaler()
poly = PolynomialFeatures()
drop_correlation = DropCorrelatedFeatures(threshold=0.9)

t_encoder = TargetEncoder()

num_pipeline = make_pipeline(scaler,poly,
                                      drop_correlation)

cat_pipeline = make_pipeline(t_encoder,scaler,
                                      KNNImputer(),
                                      poly,
                                      drop_correlation)
                                      

column_transformer = make_column_transformer((num_pipeline, num_columns),
                                              (cat_pipeline, cat_colunmns),
                                              remainder='passthrough')

column_transformer

ColumnTransformer(remainder='passthrough',
                  transformers=[('pipeline-1',
                                 Pipeline(steps=[('standardscaler',
                                                  StandardScaler()),
                                                 ('polynomialfeatures',
                                                  PolynomialFeatures()),
                                                 ('dropcorrelatedfeatures',
                                                  DropCorrelatedFeatures(threshold=0.9))]),
                                 ['RegistrationYear', 'Power', 'Kilometer',
                                  'RegistrationMonth']),
                                ('pipeline-2',
                                 Pipeline(steps=[('targetencoder',
                                                  TargetEncoder()),
                                                 ('standardscaler',
                                                  StandardScaler()),
                                                 ('knnimputer', KNNImputer()),
                                                 ('polynomialfeatures',
                                                  PolynomialFeatures()),
                                                 ('dropcorrelatedfeatures',
                                                  DropCorrelatedFeatures(threshold=0.9))]),
                                 ['VehicleType', 'Gearbox', 'Model', 'FuelType',
                                  'Brand', 'Repaired'])])

В качестве основных моделей выберем модель решающих деревьев, LightGBM и Catboost.

In [28]:
tree = DecisionTreeRegressor(random_state=42)  

lgbm = LGBMRegressor(random_state=42, verbose=0, force_row_wise=True)  

cat = CatBoostRegressor(random_state=42, verbose=0)

In [29]:
pipeline = make_pipeline(column_transformer,
                                  DropConstantFeatures(),
                                  VotingRegressor(estimators=[('tree', tree),
                                                              ('lgbm', lgbm),
                                                              ('cat', cat),
                                                                ]))
# Присваюваю имя на -1 шаг ансабль      
pipeline.steps[-1] = ('ensemble', pipeline.steps[-1][1])

pipeline                   

Pipeline(steps=[('columntransformer',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('pipeline-1',
                                                  Pipeline(steps=[('standardscaler',
                                                                   StandardScaler()),
                                                                  ('polynomialfeatures',
                                                                   PolynomialFeatures()),
                                                                  ('dropcorrelatedfeatures',
                                                                   DropCorrelatedFeatures(threshold=0.9))]),
                                                  ['RegistrationYear', 'Power',
                                                   'Kilometer',
                                                   'RegistrationMonth']),
                                                 ('pipeline-2',
                                                  Pipe...
                                                                   DropCorrelatedFeatures(threshold=0.9))]),
                                                  ['VehicleType', 'Gearbox',
                                                   'Model', 'FuelType', 'Brand',
                                                   'Repaired'])])),
                ('dropconstantfeatures', DropConstantFeatures()),
                ('ensemble',
                 VotingRegressor(estimators=[('tree',
                                              DecisionTreeRegressor(random_state=42)),
                                             ('lgbm',
                                              LGBMRegressor(force_row_wise=True,
                                                            random_state=42,
                                                            verbose=0)),
                                             ('cat',
                                              <catboost.core.CatBoostRegressor object at 0x000001F4B3884250>)]))])

Зададим подбираемые гиперпараметры для выбранных моделей.

In [30]:
parameters = {
   'ensemble__tree__max_depth': [6, 9],
    
   'ensemble__lgbm__num_leaves': [18, 36],
   'ensemble__lgbm__max_depth': [ 6, 9],
   'ensemble__lgbm__n_estimators':[500, 1000],
  
#   'ensemble__cat__iterations': [50, 100, 200],
#   'ensemble__cat__learning_rate': [0.03, 0.1],   
#   'ensemble__cat__depth': [3, 6, 9]

}



Выпоним подбор гиперпарметров модели используя RandomizedSearchCV

In [31]:
%%time
random_search = RandomizedSearchCV(pipeline, parameters, cv=3, scoring='neg_mean_squared_error', n_jobs=-1, random_state=42);

CPU times: total: 0 ns
Wall time: 0 ns


In [32]:
%%time
random_search.fit(features_train, target_train);

[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
CPU times: total: 2min 1s
Wall time: 2h 18min 56s


Рассчитаем   метрику качества  RMSE по предсказаниям ансамбля трех моделей лучшими гипер параметрами

In [33]:
random_search.best_estimator_

Pipeline(steps=[('columntransformer',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('pipeline-1',
                                                  Pipeline(steps=[('standardscaler',
                                                                   StandardScaler()),
                                                                  ('polynomialfeatures',
                                                                   PolynomialFeatures()),
                                                                  ('dropcorrelatedfeatures',
                                                                   DropCorrelatedFeatures(threshold=0.9))]),
                                                  ['RegistrationYear', 'Power',
                                                   'Kilometer',
                                                   'RegistrationMonth']),
                                                 ('pipeline-2',
                                                  Pipe...
                                                   'Repaired'])])),
                ('dropconstantfeatures', DropConstantFeatures()),
                ('ensemble',
                 VotingRegressor(estimators=[('tree',
                                              DecisionTreeRegressor(max_depth=9,
                                                                    random_state=42)),
                                             ('lgbm',
                                              LGBMRegressor(force_row_wise=True,
                                                            max_depth=9,
                                                            n_estimators=1000,
                                                            num_leaves=36,
                                                            random_state=42,
                                                            verbose=0)),
                                             ('cat',
                                              <catboost.core.CatBoostRegressor object at 0x000001F4CDBE7D10>)]))])

In [34]:
random_search.best_params_

{'ensemble__tree__max_depth': 9,
 'ensemble__lgbm__num_leaves': 36,
 'ensemble__lgbm__n_estimators': 1000,
 'ensemble__lgbm__max_depth': 9}

Метрика качества RMSE для ансамбля моделей на тренировочных данных 

In [35]:
np.sqrt(-random_search.best_score_) 

1541.6793641763438

Среднее время в секундах, затраченное на обучение модели для лучшей комбинации параметров:

In [36]:
random_search.cv_results_['mean_fit_time'][random_search.best_index_]

276.687092145284

Среднее время в секундах, затраченное на  предсказание модели

In [37]:
random_search.cv_results_['mean_score_time'][random_search.best_index_]

2.7428863843282065

## Обучение моделей данных выводы
В  Обучение моделей  реализован процесс обучения ансамбля из трех моделей: решающих деревьев, LightGBM и Catboost, для задачи регрессии. Загружены и подготовлены обучающие и целевые признаки из датасета

Созданы два пайплайна: один для числовых (num_pipeline) и один для категориальных признаков (cat_pipeline).

Пайплайн включают в себя стандартизацию, генерацию полиномиальных признаков, устранение мультиколлинеарности и заполнение пропущенных значений методом ближайших соседей (KNNImputer).

Создан ColumnTransformer, который применяет соответствующий пайплайн к числовым и категориальным признакам.

Выбраны основные модели для ансамбля: решающие деревья DecisionTreeRegressor, LGBMRegressor и CatBoostRegressor.
Использован RandomizedSearchCV для подбора лучших гиперпараметров. Обучение моделей проведено с использованием кросс-валидации (cv=3) и оценки метрики RMSE.

Рассчитана метрика качества RMSE для ансамбля моделей на тренировочных данных показала результат RMSE: 1541.68,  что меньше требуемого значения метрики 2500 в задании.

Среднее время в секундах, затраченное на обучение модели для лучшей комбинации параметров: 276.68 s.

Среднее время в секундах, затраченное на  предсказание модели 2.74 s


## Анализ моделей

Проверим  значения значении  метрики  RMSE тестовой выборке.

In [38]:
%%time
test_predictions = random_search.best_estimator_.predict(features_test)

CPU times: total: 5.08 s
Wall time: 2.37 s


In [39]:
mse = mean_squared_error(target_test, test_predictions)

Метрика качества RMSE для ансамбля моделей на тестовых данных 

In [40]:
np.sqrt(mse) 

1511.7256073484232

Рассчитана метрика качества RMSE для ансамбля моделей на тестовых данных показала результат RMSE: 1511.72,  что меньше требуемого значения метрики 2500 в задании.


## Анализ моделей выводы

Анализ моделей DecisionTreeRegressor, LGBMRegressor и CatBoostRegressor 

DecisionTreeRegressor:
Простота и интерпретируемость: Решающие деревья легко интерпретировать и визуализировать.
Быстрые в обучении и предсказании: 
Решающие деревья могут легко переобучиться.
Имеют неустойчивость к изменениям в данных.

LGBMRegressor:
Имеют высокую производительность и высокую скорость обучения, использование гистограммного метода обучения деревьев.
Эффективна в работе большим количеством признаков.
Недостатком LGBMRegressor  является чувствительность к выбросам: 

CatBoostRegressor:
Устойчивость к переобучению: CatBoost благодаря механизмам регуляризации.
Отсутствие необходимости настройки гиперпараметров: CatBoost по умолчанию предоставляет хорошие результаты без необходимости тонкой настройки гиперпараметров.
Время обучения: CatBoost может потребовать больше времени для обучения по сравнению с другими алгоритмами, особенно при больших объемах данных.


## Общие выводы по проекту 


Сервис по продаже автомобилей с пробегом «Не бит, не крашен» разрабатывает приложение для привлечения новых клиентов. В нём можно быстро узнать рыночную стоимость своего автомобиля. 

Необходимо подготовить модель с качеством предсказания метрики качества RMSE менее 2500.

В результате работы был построен ансамбль моделей с значением метрики качества  RMSE 1511.


## Выводы о подготовке данных

В ходе подготовки данных был рассмотрен dataframe компании  «Не бит, не крашен».
После первичного анализа данных исключили  не целевые признаки DateCreated, NumberOfPictures, PostalCode, LastSeen из таблицы.
Исключили из данных все объявления со стоимостью менее 250 евро, так как данная цена менее цены утилизации автомобиля.
Исключили из данных все объявления с регистаристарцией автомобилей до 1990 года и позже 2016.
Исключили из данных объявления в которых мощность  автомобиля  менее 50лс и выше 500 лс. 
Привели значения 'gasoline', 'petrol' к одному значению 'petrol', так как все это является бензином.
Приведем значения 'land_rover' к 'rover' к 'rover' так как 'land rover' - это модель марки 'rover'.
Отсутствующие данные в  Repaired заменим на 'unknown' так как не возможно восстановить информацию по данному параметру.


## Обучение моделей данных выводы
В  Обучение моделей  реализован процесс обучения ансамбля из трех моделей: решающих деревьев, LightGBM и Catboost, для задачи регрессии. Загружены и подготовлены обучающие и целевые признаки из датасета

Созданы два пайплайна: один для числовых (num_pipeline) и один для категориальных признаков (cat_pipeline).

Пайплайн включают в себя стандартизацию, генерацию полиномиальных признаков, устранение мультиколлинеарности и заполнение пропущенных значений методом ближайших соседей (KNNImputer).

Создан ColumnTransformer, который применяет соответствующий пайплайн к числовым и категориальным признакам.

Выбраны основные модели для ансамбля: решающие деревья DecisionTreeRegressor, LGBMRegressor и CatBoostRegressor.
Использован RandomizedSearchCV для подбора лучших гиперпараметров. Обучение моделей проведено с использованием кросс-валидации (cv=3) и оценки метрики RMSE.

Рассчитана метрика качества RMSE для ансамбля моделей на тренировочных данных показала результат RMSE: 1541.68,  что меньше требуемого значения метрики 2500 в задании.

Среднее время в секундах, затраченное на обучение модели для лучшей комбинации параметров: 276.68 s.

Среднее время в секундах, затраченное на  предсказание модели 2.74 s

## Анализ моделей выводы

Рассчитана метрика качества RMSE для ансамбля моделей на тестовых данных показала результат RMSE: 1511.72,  что меньше требуемого значения метрики 2500 в задании.


Анализ моделей DecisionTreeRegressor, LGBMRegressor и CatBoostRegressor 

DecisionTreeRegressor:
Простота и интерпретируемость: Решающие деревья легко интерпретировать и визуализировать.
Быстрые в обучении и предсказании: 
Решающие деревья могут легко переобучиться.
Имеют неустойчивость к изменениям в данных.

LGBMRegressor:
Имеют высокую производительность и высокую скорость обучения, использование гистограммного метода обучения деревьев.
Эффективна в работе большим количеством признаков.
Недостатком LGBMRegressor  является чувствительность к выбросам: 

CatBoostRegressor:
Устойчивость к переобучению: CatBoost благодаря механизмам регуляризации.
Отсутствие необходимости настройки гиперпараметров: CatBoost по умолчанию предоставляет хорошие результаты без необходимости тонкой настройки гиперпараметров.
Время обучения: CatBoost может потребовать больше времени для обучения по сравнению с другими алгоритмами, особенно при больших объемах данных.
